
![](https://wpguynews.com/wp-content/uploads/2021/03/content-marketing-examples-604a0b84e432c.png)
<h1><center><b>TPS - June 2022</b></center></h1> 

# <h1 style ='background-color:#f54707'>Importing Libraries</h1>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from lightgbm import LGBMRegressor
from xgboost import *
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import warnings 
from tensorflow.keras.metrics import *
warnings.filterwarnings('ignore')

In [ ]:
# Reading input file
Train_file = pd.read_csv('../input/tabular-playground-series-jun-2022/data.csv')
Train_file.head(3)

In [ ]:
# EDA pt.1
Train_file.info()

In [ ]:
# EDA pt.2
Train_file.describe()

In [ ]:
# Reading Sample Submission file
Sample = pd.read_csv("../input/tabular-playground-series-jun-2022/sample_submission.csv", index_col='row-col')
Sample.head(3)

In [ ]:
Train_file.isnull().sum().sort_values(ascending=True)

In [ ]:
cols = Train_file.columns[1:].to_list()
cols[0]

In [ ]:
"""Counting Null values per row"""
Null_count = []

for i in range(1000000):
    Null_count.append( Train_file.loc[[i]].isna().sum().sum() )

Train_file.insert(81, "Null_count", Null_count)    
Train_file

# <h1 style ='background-color:#f54707'>Visualization Part_1</h1>

In [ ]:
i = 1
plt.figure()
fig, ax = plt.subplots(10, 5,figsize=(24, 28))
for feature in cols[0:50]:
    plt.subplot(10, 5,i)
    sns.histplot(Train_file[feature],color="green", kde=True,bins=100, label=feature)
    plt.xlabel(feature, fontsize=9); plt.legend()
    i += 1
plt.show()

In [ ]:
i = 1
plt.figure()
fig, ax = plt.subplots(6, 5, figsize=(24,28))
for feature in cols[50:]:
    plt.subplot(6, 5, i)
    sns.histplot(Train_file[feature], kde=True, color='Yellow', bins=100, label=feature)
    plt.xlabel(feature, fontsize=9);
    plt.legend()
    i+=1
plt.show()

In [ ]:
Columns = pd.DataFrame(Train_file.columns.values, columns = ['Col_names'])
Nulls = pd.DataFrame(Train_file.isnull().sum().values * 100 / (len(Train_file)) , columns = ['Null_Qantity Percentage'])
Nulls = pd.concat([Columns,Nulls],axis=1)
Nulls

In [ ]:
# for index in Train_file.index : 
# #     Train_file.loc[[index]].isna().sum().sum()
#     print(f"1+{index}")

In [ ]:
# Train_file.index

# <h1 style ='background-color:#f54707'>Visualization Part_2</h1>

In [ ]:
# Checking the Null Values Distribution
sns.displot(Nulls, kind='kde', x='Null_Qantity', fill=True);

# <h1 style ='background-color:#f54707'>Visualization Part_3</h1>

In [ ]:
import plotly.graph_objects as go

F_1, F_2, F_3, F_4 = cols[:15], cols[15:40], cols[40:65], cols[65:]
k = 1 

fig = go.Figure()
fig.add_trace(go.Bar(name="Sum of total null values in F_1's",y=[Train_file[cols[:15]].isnull().sum().sum()]))
fig.add_trace(go.Bar(name="Sum of total null values in F_2's",y=[Train_file[cols[15:40]].isnull().sum().sum()]))
fig.add_trace(go.Bar(name="Sum of total null values in F_3's",y=[Train_file[cols[40:65]].isnull().sum().sum()]))
fig.add_trace(go.Bar(name="Sum of total null values in F_4's",y=[Train_file[cols[65:]].isnull().sum().sum()]))

fig.show()


In [ ]:
# For Further Training ... 
F = [[], [], [], [], []]
"""Splitting each F_i in seprate sets"""
for col in cols:
    for i in [1, 2, 3, 4]:
        if col.split('_')[1] == str(i):
            F[i].append(col)

df = [[], [], [], [], []]
"""Merging them all in one List to iterate over"""
for i in [1, 2, 3, 4]:
    df[i] = Train_file[F[i]]

In [ ]:
df[4] = pd.concat([df[4],Train_file['Null_count']], axis=1)
df[3] = pd.concat([df[3],Train_file['Null_count']], axis=1)
df[2] = pd.concat([df[2],Train_file['Null_count']], axis=1)
df[1] = pd.concat([df[1],Train_file['Null_count']], axis=1)

# <h1 style ='background-color:#f54707'>Visualization Part_4</h1>

In [ ]:
Not_Normal = cols[15:40]
k = 1 
plt.figure()
fig, ax = plt.subplots(5, 5, figsize=(22,26) )
for feature in Not_Normal:
    plt.subplot(5, 5, k)
    plt.boxplot(Train_file[feature])
    plt.xlabel(feature, fontsize=14, color='r')
    k+=1
plt.show();

# <h1 style ='background-color:#f54707'>Visualization Part_5</h1>

In [ ]:
corr_fig_two = Train_file[cols[:15]].corr()
plt.figure(figsize=(16,16));
sns.heatmap(corr_fig_two, cmap='Blues', annot=True, annot_kws={'size':8});

In [ ]:
corr_fig_two = Train_file[cols[40:55]].corr()
plt.figure(figsize=(16,16));
sns.heatmap(corr_fig_two, cmap='Blues', annot=True, annot_kws={'size':8});

In [ ]:
corr_fig_two = Train_file[cols[-15:]].corr()
plt.figure(figsize=(16,16))
sns.heatmap(corr_fig_two, cmap='Blues', annot=True, annot_kws={'size':11});

In [ ]:
corr_fig_two = Train_file[cols[15:40]].corr()
plt.figure(figsize=(16,16));
sns.heatmap(corr_fig_two, cmap='Blues', annot=True, annot_kws={'size':8});

# <h1 style ='background-color:#f54707'>Visualization Part_6</h1>

In [ ]:
cols.append("Null_count")
corr_values

In [ ]:
"""For further visualization task"""
chars = ['F_']
def remove_words(in_list, char_list):
    new_list = []
    for line in in_list:
        new_words = line.replace('F_','')
        new_list.append(new_words)
    return new_list

Viz_cols = remove_words(cols, chars)

In [ ]:
x = Viz_cols
y = Null_count_corr
fig, ax = plt.subplots(1, 1, figsize=(40,22))
ax.stem(x, y)
ax.set(xticks=Viz_cols, ylim=(min(Null_count_corr), max(Null_count_corr)), yticks=Null_count_corr)
plt.title('ABS correlation value between Column count and other features', fontsize=35)

plt.show()

**Histogram-based Gradient Boosting Regression Tree**

This estimator has native support for missing values (NaNs). During training, the tree grower learns at each split point whether samples with missing values should go to the left or right child, based on the potential gain. When predicting, samples with missing values are assigned to the left or right child consequently.
If no missing values were encountered for a given feature during training, then samples with missing values are mapped to whichever child has the most samples.

This implementation is inspired by LightGBM.

In [ ]:
df[4]

# <h1 style ='background-color:#f54707'>Modeling</h1>

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor
""" So now we are going to use Regreesion For prediction of
    the Missing Values based on training a model on present values """

for i in [4]:
    temp_df = pd.DataFrame()
    temp_df = df[i].copy()
    # The second one is for holding test values
    temp_df_1 = pd.DataFrame()
    temp_df_1 = df[i].copy() 
    # Create two diffrent data frames to hold train/test columns
    col_train = pd.DataFrame()
    col_test = pd.DataFrame()
    for col in temp_df.columns:
        Null_qt = temp_df[col].isnull().sum()
        if Null_qt == 0:
            print(f'{col} has Zero missing values ...')
        else :
            print(f'{col} has {Null_qt} missing values')
            NaN_idx = temp_df[temp_df[col].isnull()].index
            # Extracting the rows with null values in coulumns as Test index and Else as Train
            train = temp_df.drop(NaN_idx, axis=0)
            test = temp_df[temp_df.index.isin(NaN_idx)]
            # Now we should wipe the columns with null values from training set or (X)
            # And set them as prediction target for each row accordingly
            X = train.drop([col],axis=1)
            y = pd.DataFrame(train[col], columns=[col])
            
            # Using Imputer to fill values
            estimator = HistGradientBoostingRegressor(verbose=0, random_state=1, max_iter=20000, validation_fraction = 0.2).fit(X,y)
            temp_df_1[col][NaN_idx] = estimator.predict(test.drop([col],axis=1))
            
    df[i]=temp_df_1.copy()
    
"""I have commented these lines because of the lower performance in comparison to the latest model"""            
            # First Model
#             model_2 = XGBRegressor(random_state=1, n_estimators=2000, metric='r2')
#             model_2.fit(X,y)
#             score_2 = model_2.score(X,y)
#             print(f'r2 score of this column with XGboost Model is : {score_2}')
#             print('========================================')            

            # Second Model
#             model_2.fit(x=X, y=y, 
#                       batch_size=16,
#                       epochs=epochs,
#                       callbacks=[
#                           early_stop,
#                           lr_scheduler])
    
#             score_2 = model_2.score(X,y)

In [ ]:
df[4].head()

In [ ]:
""" I have used simple imputer for rest of them(F_1,2,3) 
    because of the low correlation between their values """
from sklearn.impute import *
imp = SimpleImputer(missing_values=np.nan, strategy = 'mean') 
for i in [1,3]:    
    df[i][:] = imp.fit_transform(df[i])

In [ ]:
for i in [1,2,3]:    
    df[i].drop(['Null_count'],axis=1,inplace=True)

Merged_Subsets = pd.concat([df[1], df[2], df[3], df[4]], axis=1)

In [ ]:
#Double checking null Values
Merged_Subsets.isnull().sum().sort_values(ascending=False)

# <h1 style ='background-color:#f54707'>Submitting</h1>

In [ ]:
for i in tqdm(Sample.index):
    row = int(i.split('-')[0])
    col = i.split('-')[1]
    Sample.loc[i, 'value'] = Merged_Subsets.loc[row, col]

Sample.head()
Sample.to_csv('submission.csv')



Sources : 
* https://scikit-learn.org/stable/modules/ensemble.html#histogram-based-gradient-boosting
* https://www.kaggle.com/code/abdulravoofshaik/top-3-solution-lgbm-mean
* https://towardsdatascience.com/how-to-handle-missing-data-8646b18db0d4
* https://medium.com/analytics-vidhya/imputing-missing-values-smartly-with-datawig-f2f6b4a07c27